In [1]:
import geemap
import ee

/home/zyang91/anaconda3/envs/geosptial/lib/python3.13/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize(project="ee-zhanchaoyang")

In [4]:
m=geemap.Map(center=(40,-100),zoom=4)
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [5]:
geometry = ee.Geometry.Polygon([
  [74.322, 14.981],
  [74.322, 14.765],
  [74.648, 14.765],
  [74.648, 14.980]
])

In [8]:
m.add_basemap('SATELLITE')

In [9]:
m.addLayer(geometry, {}, "geometry")
m.centerObject(geometry)

In [10]:
startDate = ee.Date.fromYMD(2022, 1, 1)

In [11]:
endDate = ee.Date.fromYMD(2023, 1, 1)

In [12]:
embeddings = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')

In [13]:
embeddingsFiltered = embeddings.filter(ee.Filter.date(startDate, endDate)).filter(ee.Filter.bounds(geometry))

## Visulization

In [17]:
embeddings_vis=embeddingsFiltered.median().clip(geometry)

In [20]:
vis_params = {
    "bands": ["A00", "A01", "A02"],
    "min": -0.5,
    "max":0.5,
}

In [21]:
m.addLayer(embeddings_vis, vis_params, "embeddings")
m

Map(bottom=240539.0, center=[14.87113268879198, 74.47151184082033], controls=(WidgetControl(options=['position…

## Tasks continue

In [14]:
# check the projections 
embeddingsProjection = ee.Image(embeddingsFiltered.first()).select(0).projection()

In [15]:
embeddingsProjection

In [22]:
embeddingsImage = embeddingsFiltered.mosaic().setDefaultProjection(embeddingsProjection)

In [23]:
gedi = ee.ImageCollection('LARSE/GEDI/GEDI04_A_002_MONTHLY')

In [24]:
def qualityMask(image):
    mask= image.updateMask(image.select('l4_quality_flag').eq(1)).updateMask(image.select('degrade_flag').eq(0))
    return mask

In [25]:
def errorMask(image):
    relative_se = image.select('agbd_se').divide(image.select('agbd'))
    error_mask= image.updateMask(relative_se.lte(0.5))
    return error_mask

In [26]:
def SlopeMask(image):
    glo30 = ee.ImageCollection('COPERNICUS/DEM/GLO30')
    glo30Filtered = glo30.filter(ee.Filter.bounds(geometry)).select('DEM')
    demProj = glo30Filtered.first().select(0).projection()
    elevation = glo30Filtered.mosaic().rename('dem').setDefaultProjection(demProj)
    slope = ee.Terrain.slope(elevation)
    slope_mask = image.updateMask(slope.lte(30))
    return slope_mask

In [27]:
gediFiltered = gedi.filter(ee.Filter.date(startDate, endDate)).filter(ee.Filter.bounds(geometry))

In [28]:
gediProjection = ee.Image(gediFiltered.first()).select('agbd').projection()

In [36]:
gedi_processed = (gediFiltered.map(qualityMask).map(errorMask).map(SlopeMask))

In [39]:
gediMosaic = gedi_processed.mosaic().select('agbd').setDefaultProjection(gediProjection)

In [45]:
gediVis = {
  min: 0,
  max: 200,
  'palette': ['#edf8fb', '#b2e2e2', '#66c2a4', '#2ca25f', '#006d2c'],
  'bands': ['agbd'],
}

In [47]:
m.addLayer(gedi_processed, gediVis, "gedi")
m

Map(bottom=240527.0, center=[14.879096301785548, 74.49417114257814], controls=(WidgetControl(options=['positio…

In [48]:
gridScale = 100

In [49]:
gridProjection = ee.Projection('EPSG:3857').atScale(gridScale)

In [50]:
stacked = embeddingsImage.addBands(gediMosaic)

In [51]:
stacked = stacked.resample('bilinear')

In [53]:
stackedResampled = (
    stacked
    .reduceResolution(
        reducer=ee.Reducer.mean(),
        maxPixels=1024
    )
    .reproject(
        crs=gridProjection
    )
)

In [54]:
stackedResampled = stackedResampled.updateMask(stackedResampled.mask().gt(0))